In [1]:
!pip install pulp
from pulp import *

In [42]:
# Creates a list of the scenarios
scenarios = ['MEDIUM', 'HIGH', 'LOW']
crops = ['Corn', 'Sugar Beets','Wheat']
products = ['Corn', 'Sugar Beets HighProfit', 'Sugar Beets LowProfit', 'Wheat']
fields = ['F1', 'F2', 'F3', 'F4']

# Dictionaries with data
fieldSizes = {'F1': 185, 
          'F2': 145,
          'F3': 105,
          'F4': 65 
}


yields = {'Wheat': 2.5, 
         'Corn': 3,
         'Sugar Beets': 20}

productCropMapping = {'Wheat': 'Wheat', 
         'Corn': 'Corn',
         'Sugar Beets HighProfit': 'Sugar Beets',
         'Sugar Beets LowProfit': 'Sugar Beets'}

maxSale = {'Wheat': -1, 
         'Corn': -1,
         'Sugar Beets HighProfit': 6000,
         'Sugar Beets LowProfit': -1
}

yieldModifier = {'HIGH': 1.2, 
         'MEDIUM': 1,
         'LOW': 0.8}

plantingCost = {'Wheat': 150, 
         'Corn': 230,
         'Sugar Beets': 260}
    
salesPrice = {'Wheat': 170, 
         'Corn': 150,
         'Sugar Beets HighProfit': 36,
         'Sugar Beets LowProfit': 10}
    
purchasePrice =  {'Wheat': 238, 
         'Corn': 210,
         'Sugar Beets': 99999}
    
requirements = {'Wheat': 200, 
         'Corn': 240,
            'Sugar Beets': 0}

# A dictionary of the probability of each scenario
probability = {'HIGH': 1/3, 
         'MEDIUM': 1/3,
         'LOW': 1/3}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Farmer Problem", LpMaximize)

In [43]:
# Create decision variables
fieldAllocation = LpVariable.dicts("fieldsAllocation",list(itertools.product(fields,crops)),cat='Binary')
plant = LpVariable.dicts("Planting",crops,0) # non-recourse variable
sales = LpVariable.dicts("Period Sales",list(itertools.product(products,scenarios)),0) # recourse variables
purchase = LpVariable.dicts("Purchase",list(itertools.product(crops,scenarios,)),0) # recourse variables

# Create objective function
prob += lpSum([probability[i]*lpSum([sales[j,i]*salesPrice[j] for j in products]) - probability[i]*lpSum([purchase[j,i]*purchasePrice[j] for j in purchasePrice.keys()]) for i in scenarios]) - lpSum([plant[i] * plantingCost[i] for i in crops])

#prob += lpSum(plant[i] for i in crops) <= 500

for j in crops:
    prob += plant[j] <= lpSum(fieldAllocation[f,j]*fieldSizes[f] for f in fields), "field allocation %s for crop %s" %(f, j)

for f in fields:
    prob += lpSum(fieldAllocation[f,j] for j in crops)<=1, "single usage fields %s" %f
    
for i in scenarios:
    for j in crops:
        # Inventory constraint
        prob += yieldModifier[i]*yields[j]*plant[j]+purchase[j,i] - lpSum(sales[p,i] for p in products if productCropMapping[p]==j) >= requirements[j], "Feed %s %s" %(i,j)
    for j in products:
        if maxSale[j]>=0:
            prob += sales[j,i]<=maxSale[j], "Sales Limit %s %s" %(j,i)
        

In [44]:
print(prob)

Farmer Problem:
MAXIMIZE
50.0*Period_Sales_('Corn',_'HIGH') + 50.0*Period_Sales_('Corn',_'LOW') + 50.0*Period_Sales_('Corn',_'MEDIUM') + 12.0*Period_Sales_('Sugar_Beets_HighProfit',_'HIGH') + 12.0*Period_Sales_('Sugar_Beets_HighProfit',_'LOW') + 12.0*Period_Sales_('Sugar_Beets_HighProfit',_'MEDIUM') + 3.333333333333333*Period_Sales_('Sugar_Beets_LowProfit',_'HIGH') + 3.333333333333333*Period_Sales_('Sugar_Beets_LowProfit',_'LOW') + 3.333333333333333*Period_Sales_('Sugar_Beets_LowProfit',_'MEDIUM') + 56.666666666666664*Period_Sales_('Wheat',_'HIGH') + 56.666666666666664*Period_Sales_('Wheat',_'LOW') + 56.666666666666664*Period_Sales_('Wheat',_'MEDIUM') + -230*Planting_Corn + -260*Planting_Sugar_Beets + -150*Planting_Wheat + -70.0*Purchase_('Corn',_'HIGH') + -70.0*Purchase_('Corn',_'LOW') + -70.0*Purchase_('Corn',_'MEDIUM') + -33333.0*Purchase_('Sugar_Beets',_'HIGH') + -33333.0*Purchase_('Sugar_Beets',_'LOW') + -33333.0*Purchase_('Sugar_Beets',_'MEDIUM') + -79.33333333333333*Purchase_('W

In [45]:
# Optimize

prob.solve()



# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])
# Print the value of the objective
print("Objective = %f" % value(prob.objective))


# Print the value of the variables at the optimum
for c in crops:
    print("%s = %f" % (plant[c].name, plant[c].varValue))

for j in sales.keys():
    print("%s %s" %(sales[j].name,sales[j].varValue))
    
for j in purchase.keys():
    if purchasePrice[j[0]] < 1000:
        print("%s %s" %(purchase[j].name,purchase[j].varValue))

    
stochasticObjective = value(prob.objective)

Status = Optimal
Objective = 107975.000000
Planting_Corn = 105.000000
Planting_Sugar_Beets = 250.000000
Planting_Wheat = 145.000000
Period_Sales_('Corn',_'MEDIUM') 75.0
Period_Sales_('Corn',_'HIGH') 138.0
Period_Sales_('Corn',_'LOW') 12.0
Period_Sales_('Sugar_Beets_HighProfit',_'MEDIUM') 5000.0
Period_Sales_('Sugar_Beets_HighProfit',_'HIGH') 6000.0
Period_Sales_('Sugar_Beets_HighProfit',_'LOW') 4000.0
Period_Sales_('Sugar_Beets_LowProfit',_'MEDIUM') 0.0
Period_Sales_('Sugar_Beets_LowProfit',_'HIGH') 0.0
Period_Sales_('Sugar_Beets_LowProfit',_'LOW') 0.0
Period_Sales_('Wheat',_'MEDIUM') 162.5
Period_Sales_('Wheat',_'HIGH') 235.0
Period_Sales_('Wheat',_'LOW') 90.0
Purchase_('Corn',_'MEDIUM') 0.0
Purchase_('Corn',_'HIGH') 0.0
Purchase_('Corn',_'LOW') 0.0
Purchase_('Wheat',_'MEDIUM') 0.0
Purchase_('Wheat',_'HIGH') 0.0
Purchase_('Wheat',_'LOW') 0.0
